In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [34]:
def scrape_listings(url):
    # Open the url
    r = requests.get(url)
    # Parse the html
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # Find all div-elements with class 
    listings = soup.find_all('div', {'class': 'ListItem_wrapper__J_a_C'})
    # Iterate over the listing
    rows = []
    for element in listings:
        row = {}
        # Find the a-element (model of car)
        headers = element.find('a', {'class': 'ListItem_title__znV2I ListItem_title_new_design__lYiAv Link_link__pjU1l'})
        row['url'] = 'https://www.autoscout24.de' + headers.get('href')
        model = headers.find('h2')
        span_elements = model.find_all('span')
        for span_element in span_elements:
            span_element.decompose() 
        row['Marke'] = "Audi"
        row['Model'] = model.text.replace("Audi", "")
        
        # Find all details about car
        details = element.find('div', {'class': 'VehicleDetailTable_container__mUUbY'})
        for detail in details:
            text = detail.text
            if(text.endswith('km')):
                row['Kilometerstand'] = text.split(' ')[0].replace('.', '')
            elif(text == "Automatik" or text == "Schaltgetriebe"):
                row['Antrieb'] = text
            elif(len(text.split('/')) == 2):
                row['Baujahr'] = text.split('/')[1]
                
        
        rows.append(row)
        
    return rows

In [4]:
r = requests.get("https://www.autoscout24.de/lst/audi?atype=C&cy=D&damaged_listing=exclude&desc=0&fregfrom=2010&fregto=2023&ocs_listing=include&powertype=kw&search_id=sphlth5ci4&sort=standard&source=homepage_search-mask&ustate=N%2CU")
# Parse the html
soup = BeautifulSoup(r.text, 'html.parser')
    
# Find all div-elements with class 
listings = soup.find_all('div', {'class': 'ListItem_wrapper__J_a_C'})

In [19]:
for element in listings:
    headers = element.find('a', {'class': 'ListItem_title__znV2I ListItem_title_new_design__lYiAv Link_link__pjU1l'})
    model = headers.find('h2')
    span_elements = model.find_all('span')
    for span_element in span_elements:
        span_element.decompose()
    print(model.text)
    

Audi A3 
Audi A4 
Audi A6 allroad 
Audi SQ5 
Audi A5 
Audi A3 
Audi A5 
Audi A5 
Audi A1 
Audi A8 
Audi A4 
Audi R8 
Audi S5 
Audi A4 
Audi Q7 
Audi A3 
Audi A4 
Audi S7 


In [35]:
df = scrape_listings("https://www.autoscout24.de/lst/audi?atype=C&cy=D&damaged_listing=exclude&desc=0&fregfrom=2010&fregto=2023&ocs_listing=include&powertype=kw&search_id=sphlth5ci4&sort=standard&source=homepage_search-mask&ustate=N%2CU!")

In [36]:
audi_df = pd.DataFrame(df)
audi_df.head()

,url,Marke,Model,Kilometerstand,Antrieb,Baujahr
0,https://www.autoscout24.de/angebote/audi-a3-1-...,Audi,A3,145300,Automatik,2010
1,https://www.autoscout24.de/angebote/audi-a4-2-...,Audi,A4,252333,Automatik,2011
2,https://www.autoscout24.de/angebote/audi-a3-2-...,Audi,A3,236082,Schaltgetriebe,2014
3,https://www.autoscout24.de/angebote/audi-a6-al...,Audi,A6 allroad,221000,Automatik,2014
4,https://www.autoscout24.de/angebote/audi-sq5-3...,Audi,SQ5,199000,Automatik,2013


In [73]:
def scrape_details(df):
    # Iterate over the rows of df
    for i in range(len(df)):
        # Open the URL
        r = requests.get(df['url'].iloc[i])
        # Parse the HTML
        soup = BeautifulSoup(r.text, 'html.parser')
        # Find the div-element with class 'DetailPage_slicesContainer__wHHae false'
        sections = soup.find_all('section')
        for section in sections:
            details_grid = section.find('div', {'class': 'DetailsSection_childrenSection__NQLD7'})
            if details_grid == None:
                continue
            # Bilgileri ekrana bastır
            dl_elements = details_grid.find_all('dl', {'class': 'DataGrid_defaultDlStyle__969Qm'})
            for dl in dl_elements:
                dt_elements = dl.find_all('dt', {'class': 'DataGrid_defaultDtStyle__yzRR_'})
                dd_elements = dl.find_all('dd', {'class': 'DataGrid_defaultDdStyle__29SKf'})
                for dt, dd in zip(dt_elements, dd_elements):
                    df[dt.text] = dd.text
    return df

In [76]:
audi_df = scrape_details(audi_df)
audi_df.to_csv('audi_uncleaned.csv')

In [77]:
audi_df.head()

,url,Marke,Model,Kilometerstand,Antrieb,Baujahr,Barzahlungspreis,Anzahlung,Laufzeit,Nettodarlehen,...,Scheckheftgepflegt,Leergewicht,Schlussrate,Nichtraucherfahrzeug,Garantie,Energieeffizienzklasse,CO₂-Effizienz,Bruttodarlehensbetrag,Bearbeitungsgebühren,Sonderzahlung
0,https://www.autoscout24.de/angebote/audi-a3-1-...,Audi,A3,126.820 km,Automatik,2011,"€ 18.999,-","€ 0,-",48 Monate,"€ 18.999,-",...,Ja,1.970 kg,"€ 27.949,-",Ja,Ja,A,Auf der Grundlage der gemessenen CO₂-Emissione...,"€ 9.895,91","€ 0,-","€ 4.500,-"
1,https://www.autoscout24.de/angebote/audi-a4-2-...,Audi,A4,126.820 km,Automatik,2011,"€ 18.999,-","€ 0,-",48 Monate,"€ 18.999,-",...,Ja,1.970 kg,"€ 27.949,-",Ja,Ja,A,Auf der Grundlage der gemessenen CO₂-Emissione...,"€ 9.895,91","€ 0,-","€ 4.500,-"
2,https://www.autoscout24.de/angebote/audi-a3-2-...,Audi,A3,126.820 km,Schaltgetriebe,2011,"€ 18.999,-","€ 0,-",48 Monate,"€ 18.999,-",...,Ja,1.970 kg,"€ 27.949,-",Ja,Ja,A,Auf der Grundlage der gemessenen CO₂-Emissione...,"€ 9.895,91","€ 0,-","€ 4.500,-"
3,https://www.autoscout24.de/angebote/audi-a6-al...,Audi,A6 allroad,126.820 km,Automatik,2011,"€ 18.999,-","€ 0,-",48 Monate,"€ 18.999,-",...,Ja,1.970 kg,"€ 27.949,-",Ja,Ja,A,Auf der Grundlage der gemessenen CO₂-Emissione...,"€ 9.895,91","€ 0,-","€ 4.500,-"
4,https://www.autoscout24.de/angebote/audi-sq5-3...,Audi,SQ5,126.820 km,Automatik,2011,"€ 18.999,-","€ 0,-",48 Monate,"€ 18.999,-",...,Ja,1.970 kg,"€ 27.949,-",Ja,Ja,A,Auf der Grundlage der gemessenen CO₂-Emissione...,"€ 9.895,91","€ 0,-","€ 4.500,-"


In [64]:
url = 'https://www.autoscout24.de/angebote/audi-s3-sportback-2-0-tfsi-quattro-pano-navi-bose-benzin-schwarz-19ab8289-e24f-4e70-8037-037f4f8d7a72'
r = requests.get(url)
# Parse the HTML
soup = BeautifulSoup(r.text, 'html.parser')
# DetailsSection_detailsSection__2cTru içindeki DetailsSection_childrenSection__NQLD7'yi bul
details_section = soup.find('div', {'class': 'DetailsSection_detailsSection__2cTru'})
children_section = details_section.find('div', {'class': 'DetailsSection_childrenSection__NQLD7'})

# Bilgileri ekrana bastır
dl_elements = children_section.find_all('dl', {'class': 'DataGrid_defaultDlStyle__969Qm'})
for dl in dl_elements:
    dt_elements = dl.find_all('dt', {'class': 'DataGrid_defaultDtStyle__yzRR_'})
    dd_elements = dl.find_all('dd', {'class': 'DataGrid_defaultDdStyle__29SKf'})
    for dt, dd in zip(dt_elements, dd_elements):
        print(f"{dt.text}: {dd.text}")

In [70]:
url = 'https://www.autoscout24.de/angebote/audi-s3-sportback-2-0-tfsi-quattro-pano-navi-bose-benzin-schwarz-19ab8289-e24f-4e70-8037-037f4f8d7a72'
r = requests.get(url)
# Parse the HTML
soup = BeautifulSoup(r.text, 'html.parser')
sections = soup.find_all('section')
for section in sections:
    details_grid = section.find('div', {'class': 'DetailsSection_childrenSection__NQLD7'})
    if details_grid == None:
        continue
    # Bilgileri ekrana bastır
    dl_elements = details_grid.find_all('dl', {'class': 'DataGrid_defaultDlStyle__969Qm'})
    for dl in dl_elements:
        dt_elements = dl.find_all('dt', {'class': 'DataGrid_defaultDtStyle__yzRR_'})
        dd_elements = dl.find_all('dd', {'class': 'DataGrid_defaultDdStyle__29SKf'})
        for dt, dd in zip(dt_elements, dd_elements):
            print(f"{dt.text}: {dd.text}")

Barzahlungspreis: € 12.499,-
Anzahlung: -
Laufzeit: 84 Monate
Nettodarlehen: € 12.499,-
Effektiver Jahreszins: 5,68 %
Sollzins geb. p.a.: 5,50 %
Monatliche Rate: € 180,-
Karosserieform: Limousine
Fahrzeugart: Gebraucht
Antriebsart: Allrad
Sitzplätze: 5
Türen: 4
Länderversion: Deutsche Ausführung
Angebotsnummer: 18827
Kilometerstand: 186.500 km
Erstzulassung: 08/2012
HU: 11/2023
Fahrzeughalter: 2
Scheckheftgepflegt: Ja
Nichtraucherfahrzeug: Ja
Leistung: 195 kW (265 PS)
Getriebe: Automatik
Hubraum: 1.984 cm³
Kraftstoff: Super 95
Kraftstoffverbrauch2: 8,4 l/100 km (komb.)11,2 l/100 km (innerorts)6,8 l/100 km (außerorts)
CO₂-Emissionen2: 195 g/km (komb.)
Energieeffizienzklasse: C
CO₂-Effizienz: Auf der Grundlage der gemessenen CO₂-Emissionen unter Berücksichtigung der Masse des Fahrzeugs ermittelt.
Schadstoffklasse: Euro 5
Umweltplakette: 4 (Grün)
Komfort: 2-Zonen-KlimaautomatikArmlehneBerganfahrassistentEinparkhilfeEinparkhilfe Sensoren hintenEinparkhilfe Sensoren vorneElektr. Fensterhebe